In [ ]:
# Q108_A: Toy political polarization (bounded confidence, network contrast)
# Single-cell Colab script, fully offline, no API key required.

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


def main():
    """
    Entry point for the Q108_A experiment.

    This version is tuned to create a clear contrast between
    a well-mixed network and a two-community network under the
    same bounded-confidence dynamics and initial opinions.

    It will:
      1) Build two types of social networks for the same population.
      2) Initialize opinions with two mildly separated groups.
      3) Run bounded-confidence opinion dynamics.
      4) Compute a scalar polarization tension T_polar.
      5) Save a CSV of per-run results and two PNG plots.
    """

    print("============================================================")
    print("Q108_A · Toy political polarization (network contrast)")
    print("============================================================")
    print("This notebook is fully offline. No API key is required.")
    print("It will:")
    print("  1) Build synthetic social networks (well_mixed vs two_communities).")
    print("  2) Initialize two opinion groups with mild separation.")
    print("  3) Run bounded-confidence dynamics with a shared epsilon grid.")
    print("  4) Compute a scalar polarization tension T_polar.")
    print("  5) Save a CSV of results and two PNG plots.\n")

    # --------------------------------------------------------------
    # 1. Global experimental settings
    # --------------------------------------------------------------
    N = 200                          # number of agents
    T_steps = 120                    # number of time steps per run
    epsilon_list = [0.15, 0.25, 0.35, 0.60]
    network_types = ["well_mixed", "two_communities"]
    n_runs = 10                      # repeats per configuration
    mu = 0.5                         # update step size toward neighbor
    base_seed = 108                  # global seed for reproducibility

    # Opinions start as two groups around -0.25 and +0.25.
    # Center zone is defined around zero for diagnostics.
    center_thresh = 0.15

    print("Global settings:")
    print(f"  N = {N} agents")
    print(f"  T = {T_steps} time steps")
    print(f"  epsilons = {epsilon_list}")
    print(f"  network_types = {network_types}")
    print(f"  runs per config = {n_runs}")
    print(f"  center_thresh = {center_thresh}")
    print("------------------------------------------------------------\n")

    rng_global = np.random.default_rng(base_seed)

    # --------------------------------------------------------------
    # 2. Network construction and initial opinions
    # --------------------------------------------------------------

    def build_network(network_type, N, rng):
        """
        Build a simple social network graph and a community label vector.

        Parameters
        ----------
        network_type : str
            Either "well_mixed" or "two_communities".
        N : int
            Number of nodes.
        rng : np.random.Generator
            Random number generator.

        Returns
        -------
        G : networkx.Graph
            The constructed graph with N nodes labeled 0..N-1.
        labels : np.ndarray of shape (N,)
            Community labels in {0, 1} for each node.
        """
        if network_type == "well_mixed":
            # Erdos-Renyi random graph with moderate edge probability.
            p_edge = 0.20
            G = nx.erdos_renyi_graph(
                N,
                p_edge,
                seed=int(rng.integers(0, 1_000_000))
            )
            # Community labels are just a 50/50 split, edges ignore them.
            labels = np.zeros(N, dtype=int)
            labels[N // 2 :] = 1

        elif network_type == "two_communities":
            # Stochastic block model with strong community structure.
            sizes = [N // 2, N - N // 2]
            p_in = 0.40
            p_out = 0.01
            p_matrix = [[p_in, p_out],
                        [p_out, p_in]]
            G = nx.stochastic_block_model(
                sizes,
                p_matrix,
                seed=int(rng.integers(0, 1_000_000))
            )
            G = nx.convert_node_labels_to_integers(G, first_label=0)

            labels = np.zeros(N, dtype=int)
            labels[sizes[0] :] = 1

        else:
            raise ValueError(f"Unknown network_type: {network_type}")

        return G, labels

    def init_opinions_from_labels(labels, rng):
        """
        Initialize opinions as two mildly separated groups.

        Nodes with label 0 start near -0.25, label 1 near +0.25.
        Both are clipped to [-1, 1].
        """
        N_local = len(labels)
        opinions = np.zeros(N_local, dtype=float)

        for i, g in enumerate(labels):
            mu_group = -0.25 if g == 0 else 0.25
            opinions[i] = rng.normal(loc=mu_group, scale=0.12)

        opinions = np.clip(opinions, -1.0, 1.0)
        return opinions

    # --------------------------------------------------------------
    # 3. Polarization metric: T_polar and diagnostics
    # --------------------------------------------------------------

    def compute_T_polar(opinions, center_thresh=0.15):
        """
        Compute a scalar polarization tension T_polar in [0, 1].

        The construction uses three groups:
          - left   : opinions < -center_thresh
          - center : |opinion| <= center_thresh
          - right  : opinions >  center_thresh

        T_polar is large when:
          - many agents sit in the extremes, and
          - the mean opinions of left and right groups are far apart, and
          - the center group is small.
        """
        x = np.asarray(opinions)
        left_mask = x < -center_thresh
        center_mask = np.abs(x) <= center_thresh
        right_mask = x > center_thresh

        N_local = len(x)
        if N_local == 0:
            return 0.0, 0.0, 0.0

        p_L = np.sum(left_mask) / N_local
        p_C = np.sum(center_mask) / N_local
        p_R = np.sum(right_mask) / N_local

        if np.sum(left_mask) > 0:
            mu_L = x[left_mask].mean()
        else:
            mu_L = 0.0

        if np.sum(right_mask) > 0:
            mu_R = x[right_mask].mean()
        else:
            mu_R = 0.0

        gap = abs(mu_R - mu_L)

        # Raw tension: extremes share * separation * lack of center.
        T_raw = (p_L + p_R) * gap * (1.0 - p_C)

        # Map into [0, 1] range with a simple cap.
        T_polar = max(0.0, min(1.0, T_raw))

        p_extreme = p_L + p_R
        return T_polar, p_C, p_extreme

    def summarize_final_state(opinions, center_thresh=0.15):
        T_polar_final, p_center, p_extreme = compute_T_polar(
            opinions,
            center_thresh=center_thresh
        )
        return {
            "T_polar_final": T_polar_final,
            "p_center_final": p_center,
            "p_extreme_final": p_extreme,
        }

    # --------------------------------------------------------------
    # 4. Opinion dynamics: bounded confidence update
    # --------------------------------------------------------------

    def bounded_confidence_step(G, opinions, epsilon, mu, rng):
        """
        Perform one asynchronous bounded-confidence update over all agents.

        For each agent i:
          - pick one random neighbor j (if any);
          - if |x_j - x_i| <= epsilon, move x_i toward x_j by a factor mu.
        """
        N_local = len(opinions)
        nodes = np.arange(N_local)
        rng.shuffle(nodes)

        for i in nodes:
            neighbors = list(G.neighbors(i))
            if not neighbors:
                continue
            j = neighbors[int(rng.integers(0, len(neighbors)))]
            diff = opinions[j] - opinions[i]
            if abs(diff) <= epsilon:
                opinions[i] = opinions[i] + mu * diff

    def run_simulation(G,
                       opinions_init,
                       epsilon,
                       mu,
                       T_steps,
                       center_thresh,
                       snapshot=False,
                       rng=None):
        """
        Run the bounded-confidence dynamics for T_steps and return diagnostics.
        """
        if rng is None:
            rng = np.random.default_rng()

        opinions = opinions_init.astype(float).copy()
        times = []
        t_polar_values = []

        for t in range(T_steps):
            bounded_confidence_step(G, opinions, epsilon, mu, rng)
            if snapshot and (t % 5 == 0 or t == T_steps - 1):
                T_val, _, _ = compute_T_polar(opinions, center_thresh=center_thresh)
                times.append(t)
                t_polar_values.append(T_val)

        trajectory = {}
        if snapshot:
            trajectory = {
                "time": np.array(times),
                "T_polar": np.array(t_polar_values),
            }

        return opinions, trajectory

    # --------------------------------------------------------------
    # 5. Parameter sweep over network_type × epsilon × run
    # --------------------------------------------------------------

    print("Running main parameter sweep...")
    results = []

    total_configs = len(network_types) * len(epsilon_list) * n_runs
    config_counter = 0

    for network_type in network_types:
        for epsilon in epsilon_list:
            for run_id in range(n_runs):
                config_counter += 1
                print(
                    f"[{config_counter}/{total_configs}] "
                    f"network={network_type}, epsilon={epsilon}, run={run_id}"
                )

                seed = int(rng_global.integers(0, 1_000_000))
                rng_run = np.random.default_rng(seed)

                G, labels = build_network(network_type, N, rng_run)
                opinions_init = init_opinions_from_labels(labels, rng_run)

                opinions_final, _ = run_simulation(
                    G,
                    opinions_init,
                    epsilon,
                    mu,
                    T_steps,
                    center_thresh=center_thresh,
                    snapshot=False,
                    rng=rng_run,
                )

                summary = summarize_final_state(
                    opinions_final,
                    center_thresh=center_thresh,
                )
                summary.update({
                    "network_type": network_type,
                    "epsilon": epsilon,
                    "run_id": run_id,
                })
                results.append(summary)

    results_df = pd.DataFrame(results)
    results_df = results_df[
        [
            "network_type",
            "epsilon",
            "run_id",
            "T_polar_final",
            "p_center_final",
            "p_extreme_final",
        ]
    ]

    print("\nFinished parameter sweep.")
    print("First few rows of raw results:")
    print(results_df.head())

    csv_path = "Q108_A_results.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"\nSaved raw results to: {csv_path}")

    # --------------------------------------------------------------
    # 6. Aggregated summary: mean and std over runs
    # --------------------------------------------------------------

    grouped = results_df.groupby(["network_type", "epsilon"])
    summary_df = grouped.agg(
        T_polar_final_mean=("T_polar_final", "mean"),
        T_polar_final_std=("T_polar_final", "std"),
        p_center_mean=("p_center_final", "mean"),
        p_extreme_mean=("p_extreme_final", "mean"),
    ).reset_index()

    print("\nAggregated summary (per network_type × epsilon):")
    print(summary_df)

    # --------------------------------------------------------------
    # 7. Plot: T_polar_final_mean vs epsilon for each network type
    # --------------------------------------------------------------

    print("\nGenerating plot: T_polar vs epsilon...")
    fig1, ax1 = plt.subplots(figsize=(6, 4))

    for network_type in network_types:
        sub = summary_df[summary_df["network_type"] == network_type]
        ax1.errorbar(
            sub["epsilon"],
            sub["T_polar_final_mean"],
            yerr=sub["T_polar_final_std"],
            marker="o",
            linestyle="-",
            label=network_type,
        )

    ax1.set_xlabel("epsilon (confidence threshold)")
    ax1.set_ylabel("T_polar (final mean over runs)")
    ax1.set_title("Q108_A · Polarization vs epsilon")
    ax1.legend()
    ax1.grid(True)

    fig1.tight_layout()
    plot_path1 = "Q108_A_polarization_vs_epsilon.png"
    fig1.savefig(plot_path1, dpi=150)
    plt.show()
    print(f"Saved figure: {plot_path1}")

    # --------------------------------------------------------------
    # 8. Example opinion distributions for a single epsilon
    # --------------------------------------------------------------

    print("\nGenerating example opinion distributions...")

    example_epsilon = 0.35
    fig2, axes = plt.subplots(
        nrows=1,
        ncols=len(network_types),
        figsize=(10, 4),
        sharey=True,
    )

    if len(network_types) == 1:
        axes = [axes]

    for ax, network_type in zip(axes, network_types):
        seed = int(rng_global.integers(0, 1_000_000))
        rng_example = np.random.default_rng(seed)

        G, labels = build_network(network_type, N, rng_example)
        opinions_init = init_opinions_from_labels(labels, rng_example)

        opinions_final, traj = run_simulation(
            G,
            opinions_init,
            example_epsilon,
            mu,
            T_steps,
            center_thresh=center_thresh,
            snapshot=True,
            rng=rng_example,
        )

        ax.hist(opinions_final, bins=20, range=(-1.0, 1.0))
        ax.set_xlabel("opinion value")
        ax.set_title(f"{network_type}\n epsilon={example_epsilon}")

    axes[0].set_ylabel("number of agents")
    fig2.suptitle("Q108_A · Example final opinion distributions")
    fig2.tight_layout(rect=[0, 0.0, 1, 0.95])

    plot_path2 = "Q108_A_opinion_distributions_examples.png"
    fig2.savefig(plot_path2, dpi=150)
    plt.show()
    print(f"Saved figure: {plot_path2}")

    print("\nAll done. You can now:")
    print("  - Inspect Q108_A_results.csv for raw values.")
    print("  - Use the two PNGs as candidates for the Q108 README.")
    print("If you want to change parameters, scroll up and adjust the values")
    print("in the 'Global experimental settings' section, then run this cell again.")


if __name__ == "__main__":
    main()
